# MATH 468 - Stochastic Calculus II - HW 4

In [1]:
'''Packages Used'''
import pandas as pd
import numpy as np

'''Code Deck Import'''
from StoCalCodeDeck import walk_params

## Problem 1 (American options with N large)

*Note*: This problem is a continuation of Problem 3 in HW #3. The objective is to generalize your code to American options. Please provide the answers to the questions below as well as a sample of your code.

We consider the same market model as in Problem 3 of HW #3. We consider an American put option with strike price $K = \$1,025$ and maturity $T = 1$ year.

(a) Find the price of the option at time $t_0 = 0$ if it is American and compare with the price of the European put found in Problem 1.  
(b) What is the earliest possible time of optimal exercise of the American put?  
(c) Repeat questions (a) and (b) in the case where $r = 0\%$. What do you conclude?  

### Solution to Part (a)

In [70]:
'''Code Block'''

def build_tree(N, T, mu, sigma, s0, k, r, option_type, nature):
    '''Establish time delta -- each variable represents
    T - time to maturity in years
    N - number of periods (i.e. time steps)'''
    dt = T / N

    '''Establish up and down factos'''
    u = np.exp(mu*dt + sigma*np.sqrt(dt))
    d = np.exp(mu*dt - sigma*np.sqrt(dt))
    
    '''Establish risk neutral probability'''
    q = ((1+r*dt) - d) / (u - d)
    
    '''Constructs the data frame -- each column and its representation
    index - number
    layer - time step
    stock_price - stock price at that node and layer
    phi - shares in replicating protfolio
    psi - amount of funds in money market
    option value - option price at that node'''
    tree = pd.DataFrame(data=None, index=range(0,sum(range(N+2))))
    tree.index.name = 'node'
    tree['layer'] = ""
    tree['stock_price'] = 0
    tree['phi'] = 0
    tree['psi'] = 0
    
    '''inputs intial stock price'''
    tree['stock_price'].loc[0] = s0

    '''Inputs layers (time steps) and further stock prices'''
    node = 0
    for i in range(0,N+2):
        if i < (N+1):
            tree.iloc[range(node,node+i+1),0] = i #Layer
            tree.iloc[range(node,node+i+1),1] = s0 * d ** (np.arange(i,-1,-1)) * u ** (np.arange(0,i+1,1)) #Stock Price
            node += i + 1

    '''Puts intrinsic option value in at every node'''
    if option_type == 'c':
        tree['option_value'] = tree['stock_price'].apply(lambda x: x-k if x-k > 0 else 0)
    elif option_type == 'p':
        tree['option_value'] = tree['stock_price'].apply(lambda x: k-x if k-x > 0 else 0)
    
    '''resets node to N-1'''
    node= node - N
    
    '''Calculates option value and replicating portfolio going backwards'''
    for i in reversed(range(N)):
        price_range = (1 / (1+r*dt)) * (q * tree.iloc[range(node,node+i+1),4].to_numpy() + (1-q) * tree.iloc[range(node-1,node+i),4].to_numpy()) # price option
        if nature == 'european':
            tree.iloc[range(node-i-2,node-1),4] = price_range
        elif nature == 'american':
            print(tree.iloc[range(node-i-2,node-1),4].to_numpy())
            tree.iloc[range(node-i-2,node-1),4] = np.maximum(price_range, tree.iloc[range(node-i-2,node-1),4].to_numpy())
        tree.iloc[range(node-i-2,node-1),2] = (tree.iloc[range(node,node+i+1),4].to_numpy() - tree.iloc[range(node-1,node+i),4].to_numpy()) / (tree.iloc[range(node,node+i+1),1].to_numpy() - tree.iloc[range(node-1,node+i),1].to_numpy()) # get delta
        tree.iloc[range(node-i-2,node-1),3] = (tree.iloc[range(node,node+i+1),4].to_numpy() - tree.iloc[range(node-i-2,node-1),2].to_numpy() * tree.iloc[range(node,node+i+1),1].to_numpy()) / (1 + r*dt) # Money Market
        node = node - i - 1


    return tree

In [72]:
T=1 # 1 Year
N=60 # 60 Periods
mu, sigma = walk_params(u=0.005,d=0.003,T=T,N=N)
tree_c = build_tree(N=N, T=T, mu=mu, sigma=sigma, s0=1000, k=1025, r=0, option_type='p', nature='american')
tree_c
#3.426341

C:\Users\jdean\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


[187.44306223 180.72244488 173.9479008  167.1189973  160.23529818
 153.29636376 146.30175083 139.25101263 132.14369879 124.97935535
 117.7575247  110.47774556 103.13955294  95.74247814  88.28604868
  80.76978829  73.19321688  65.55585051  57.85720137  50.09677771
  42.27408385  34.38862013  26.43988288  18.42736439  10.35055287
   2.20893243   0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.        ]
[184.92283072 178.18199085 171.38706199 164.53761013 157.63319777
 150.67338391 143.65772401 136.58576994 129.45707    122.27116885
 115.02760752 107.72592333 100.36564989  92.94631709  85.46745103
  77.92857401  70.32920449  62.66885709  54.9470425   47.

,layer,stock_price,phi,psi,option_value
node,,,,,
0,0,1000.000000,-0.782087,810.688800,28.602071
1,1,997.000000,-0.812010,840.521914,30.948331
2,1,1005.000000,-0.732612,760.966942,24.691637
3,2,994.009000,-0.839664,868.010288,33.377052
4,2,1001.985000,-0.766286,794.707959,26.900463
...,...,...,...,...,...
1886,60,1306.411791,0.000000,0.000000,0.000000
1887,60,1316.894534,0.000000,0.000000,0.000000
1888,60,1327.461391,0.000000,0.000000,0.000000


In [74]:
tree_c = build_tree(N=N, T=T, mu=mu, sigma=sigma, s0=1000, k=1025, r=0, option_type='p', nature='european')
tree_c

C:\Users\jdean\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,layer,stock_price,phi,psi,option_value
node,,,,,
0,0,1000.000000,-0.782087,810.688800,28.602071
1,1,997.000000,-0.812010,840.521914,30.948331
2,1,1005.000000,-0.732612,760.966942,24.691637
3,2,994.009000,-0.839664,868.010288,33.377052
4,2,1001.985000,-0.766286,794.707959,26.900463
...,...,...,...,...,...
1886,60,1306.411791,0.000000,0.000000,0.000000
1887,60,1316.894534,0.000000,0.000000,0.000000
1888,60,1327.461391,0.000000,0.000000,0.000000


### Solution to Part (b)

In [3]:
'''Code Block'''

'Code Block'

### Solution to Part (c)

In [4]:
'''Code Block'''

'Code Block'

## Problem 2 (Option pricing from real data.)

*Note*: This problem is the continuation of Prob. 3 in HW #1. We will use the data to price options and compare with the market prices.

Consider the asset that you selected in HW #1. For this asset, select three call options and one put option. All options should have approximately the same expiration date, say about 3 months in the future. The three calls should have three different strike prices, typically one *in-the-money*, one *at-the-money*, and one *out-of-the-money*. (Consider values relatively close to the current price and, if you have access to this information, for which the trading volume is relatively large.) The put option should be *at-the-money*. Record the current price of these options. Note that all traded options are American options. Also determine the interest rate to be used for your calculations: typically, you can consider the LIBOR rate, or the 10-year US treasury yield.

(a) We assume that the CRR model represents the stock price and that its assumptions are satisfied. Based on historical data (about 3 months to a year in the past), determine the parameters $\mu$ and $\sigma$ for the stock that you chose following the methods discussed in class. (I recommend to download current data, but feel free to use past data if that is all you have.)  
(b) Price the selected put option using an $N$-period binomial model for different values of $N$ (which you can choose on your own). As a hint, you can for instance start with $N = 2$ or $N = 3$, which should be approximately monthly-periods. You can then use approximately weekly periods ($N \approx 10$ or $N \approx 12$) or daily periods ($N \approx 60$). Make sure that you use the exact number of days or weeks to the maturity of your option.  
(c) Compare your results to the market price and comment. In particular, discuss the impact of $N$ on your calculated price.  
(d) Price each of the three selected call options using all of (i) approximately monthly periods ($N = 2, 3$), (ii) approximately weekly periods ($N \approx 10, 12$), and (iii) daily periods ($N \approx 60$).  
(e) Compare your results to the market prices and comment. In particular, discuss the impact of the strike price $K$ on your results. In general, once you obtain the parameters $\mu$ and $\sigma$, you should be able to use the computing tool that you developed in HW#3 and in Problem 1 to price your options.


### Solution to Part (a)

In [5]:
'''Code Block'''

'Code Block'

### Solution to Part (b)

In [6]:
'''Code Block'''

'Code Block'

### Solution to Part (c)

In [7]:
'''Code Block'''

'Code Block'

### Solution to Part (d)

In [8]:
'''Code Block'''

'Code Block'

### Solution to Part (e)

In [9]:
'''Code Block'''

'Code Block'

## Problem 3

We consider an N-period binomial model with the following properties: each period lasts a year and the current stock price is S0 = 4. On each period, the stock price doubles when it moves up and is reduced by half when it moves down. The annual interest rate on the money market is $25\%$.

(a) Find the values of $\mu$ and $\sigma$ associated with this CRR model.  
(b) Assuming that $p = \frac{1}{2}$, find $P(S_T > 6)$ when $N = 3$ and $N = 10$.  
(c) Using the risk-neutral option pricing formula (written as an expectation), calculate the price of a European call with strike price $K = 8$ when $N = 3$ and $N = 10$.


### Solution to Part (a)

In [10]:
'''Code Block'''

'Code Block'

### Solution to Part (b)

In [11]:
'''Code Block'''

'Code Block'

### Solution to Part (c)

In [12]:
'''Code Block'''

'Code Block'